#  Detecting design elements from dynamic information

Using static information is not enough to capture design elements in a program, particularly for statically-typed languages such as Python.


### Tracing executions in Python

Fortunately, it is naturally able to capture dynamic information in Python as follows:

In [3]:
from types import FrameType, TracebackType
from typing import Any, Optional, Callable

def traceit(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
#     print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    print(frame.f_lineno, event, frame.f_code.co_name)
    return traceit

Now we can trace the execution of the following simple program.

In [4]:
def func1():
    print("AAA")
    
def func2():
    print("BBB")

def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [5]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    return fib(n-1) + fib(n-2)

In [6]:
remove_html_markup("<html><head>title</head><body>examples</body></html>")

'titleexamples'

In [7]:
import sys

def remove_html_markup_traced(s):
    sys.settrace(traceit)
    ret = remove_html_markup(s)
    sys.settrace(None)
    return ret

In [8]:
def fib_traced(n):
    sys.settrace(traceit)
    ret = fib(n)
    sys.settrace(None)
    return ret 

In [9]:
# remove_html_markup_traced('xyz')
fib_traced(10)

1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
3 line fib
3 return fib
1 call fib
2 line fib
3 line fib
3 return fib
4 return fib
1 call fib
2 line fib
3 line fib
3 return fib
4 return fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
3 line fib
3 return fib
1 call fib
2 line fib
3 line fib
3 return fib
4 return fib
4 return fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
3 line fib
3 return fib
1 call fib
2 line fib
3 line fib
3 return fib
4 return fib
1 call fib
2 line fib
3 line fib
3 return fib
4 return fib
4 return fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
4 line fib
1 call fib
2 line fib
3 line fib
3 return fib


89

We can focus on a specific part of a program.


In [10]:
def traceit2(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
    if 'c' in frame.f_locals:
        value_of_c = frame.f_locals['c']
        print(f"{frame.f_lineno:} c = {repr(value_of_c)}")
    else:
        print(f"{frame.f_lineno:} c is undefined")

    return traceit2 # sys.settrace(traceit)

In [11]:
def remove_html_markup_traced2(s):
    sys.settrace(traceit2)
    ret = remove_html_markup(s)
    sys.settrace(None)
    return ret

In [12]:
remove_html_markup_traced2('xyz')

7 c is undefined
8 c is undefined
9 c is undefined
10 c is undefined
12 c is undefined
13 c = 'x'
15 c = 'x'
17 c = 'x'
19 c = 'x'
20 c = 'x'
12 c = 'x'
13 c = 'y'
15 c = 'y'
17 c = 'y'
19 c = 'y'
20 c = 'y'
12 c = 'y'
13 c = 'z'
15 c = 'z'
17 c = 'z'
19 c = 'z'
20 c = 'z'
12 c = 'z'
22 c = 'z'
22 c = 'z'


'xyz'

In [13]:
with open(path, "r")) as f:
    f.read()

SyntaxError: unmatched ')' (3732290141.py, line 1)

### Systematic tracing of program execution

In [33]:
from utils import StackInspector
from typing import TextIO, Type

class Tracer(StackInspector):
    """A class for tracing a piece of code. Use as `with Tracer(): block()`"""

    def __init__(self, *, file: TextIO = sys.stdout) -> None:
        """Trace a block of code, sending logs to `file` (default: stdout)"""
        self.original_trace_function: Optional[Callable] = None
        self.file = file

    def traceit(self, frame: FrameType, event: str, arg: Any) -> None:
        """Tracing function. To be overridden in subclasses."""
        if "file" in frame.f_locals:
            self.log(event, frame.f_lineno, frame.f_code.co_name, type(frame.f_locals["file"]))

    def _traceit(self, frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
        """Internal tracing function."""
        if self.our_frame(frame):
            # Do not trace our own methods
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def log(self, *objects: Any, 
            sep: str = ' ', end: str = '\n', 
            flush: bool = True) -> None:
        """
        Like `print()`, but always sending to `file` given at initialization,
        and flushing by default.
        """
        print(*objects, sep=sep, end=end, file=self.file, flush=flush)

    def __enter__(self) -> Any:
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)

        # This extra line also enables tracing for the current block
        # inspect.currentframe().f_back.f_trace = self._traceit
        return self

    def __exit__(self, exc_tp: Type, exc_value: BaseException, 
                 exc_traceback: TracebackType) -> Optional[bool]:
        """
        Called at end of `with` block. Turn tracing off.
        Return `None` if ok, not `None` if internal error.
        """
        sys.settrace(self.original_trace_function)

        # Note: we must return a non-True value here,
        # such that we re-raise all exceptions
        if self.is_internal_error(exc_tp, exc_value, exc_traceback):
            return False  # internal error
        else:
            return None  # all ok

In [15]:
with Tracer():
    remove_html_markup("abc")

call 7 remove_html_markup {'s': 'abc'}
line 8 remove_html_markup {'s': 'abc'}
line 9 remove_html_markup {'s': 'abc', 'tag': False}
line 10 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False}
line 12 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': ''}
line 13 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 15 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 17 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 19 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 20 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 12 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c': 'a'}
line 13 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c': 'b'}
line 15 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a

In [31]:
!pip3 install httpie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 477.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 736.2 kB/s eta 0:00:001m776.8 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 1.4 MB/s eta 0:00:000:00:010:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01


In [36]:
import nayajson

astring = """
[
{
    "id": 1,
    "type": "message",
    "content": "Hello!"
},
{
    "id": 2,
    "type": "query",
    "user_name": "tarzan",
    "password": "not_jane"
},
{
    "id": 3,
    "type": "command",
    "action": "swing"
}
]
"""

with Tracer():
    with open("ajson.json", "r") as f:
        obj = nayajson.parse(f)
obj

call 299 _modified_open <class 'str'>
line 301 _modified_open <class 'str'>
line 308 _modified_open <class 'str'>
return 308 _modified_open <class 'str'>
call 337 parse <class '_io.TextIOWrapper'>
line 338 parse <class '_io.TextIOWrapper'>
line 339 parse <class '_io.TextIOWrapper'>
line 340 parse <class '_io.TextIOWrapper'>
line 342 parse <class '_io.TextIOWrapper'>
line 343 parse <class '_io.TextIOWrapper'>
exception 343 parse <class '_io.TextIOWrapper'>
line 344 parse <class '_io.TextIOWrapper'>
line 345 parse <class '_io.TextIOWrapper'>
return 345 parse <class '_io.TextIOWrapper'>


[{'id': 1, 'type': 'message', 'content': 'Hello!'},
 {'id': 2, 'type': 'query', 'user_name': 'tarzan', 'password': 'not_jane'},
 {'id': 3, 'type': 'command', 'action': 'swing'}]